In [1]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE


In [2]:

# Directories
train_directories = [
    './Documents/Research/can-train-and-test/set_01/train_01/',
    './Documents/Research/can-train-and-test/set_02/train_01/',
    './Documents/Research/can-train-and-test/set_03/train_01/',
    './Documents/Research/can-train-and-test/set_04/train_01/'
]

# Test files for DoS attack
dos_files = [
    './Documents/Research/can-train-and-test/set_01/test_01_known_vehicle_known_attack/DoS-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_01_known_vehicle_known_attack/DoS-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_02_unknown_vehicle_known_attack/DoS-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_02_unknown_vehicle_known_attack/DoS-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_03_known_vehicle_unknown_attack/DoS-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_03_known_vehicle_unknown_attack/DoS-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_04_unknown_vehicle_unknown_attack/DoS-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_04_unknown_vehicle_unknown_attack/DoS-4.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/DoS-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/DoS-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/DoS-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/DoS-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/DoS-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/DoS-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/DoS-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/DoS-2.csv'
]

# Test files for Force neutral attack
fn_files = [
    './Documents/Research/can-train-and-test/set_01/test_01_known_vehicle_known_attack/force-neutral-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_01_known_vehicle_known_attack/force-neutral-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_02_unknown_vehicle_known_attack/force-neutral-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_02_unknown_vehicle_known_attack/force-neutral-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_03_known_vehicle_unknown_attack/force-neutral-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_03_known_vehicle_unknown_attack/force-neutral-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_04_unknown_vehicle_unknown_attack/force-neutral-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_04_unknown_vehicle_unknown_attack/force-neutral-4.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/force-neutral-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/force-neutral-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/force-neutral-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/force-neutral-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/force-neutral-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/force-neutral-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/force-neutral-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/force-neutral-2.csv'
]

# Test files for RPM attack
rpm_files = [
    './Documents/Research/can-train-and-test/set_01/test_01_known_vehicle_known_attack/rpm-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_01_known_vehicle_known_attack/rpm-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_02_unknown_vehicle_known_attack/rpm-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_02_unknown_vehicle_known_attack/rpm-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_03_known_vehicle_unknown_attack/rpm-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_03_known_vehicle_unknown_attack/rpm-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_04_unknown_vehicle_unknown_attack/rpm-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_04_unknown_vehicle_unknown_attack/rpm-4.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/rpm-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/rpm-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/rpm-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/rpm-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/rpm-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/rpm-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/rpm-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/rpm-2.csv'
]

# Test files for standstill attack
standstill_files = [
    './Documents/Research/can-train-and-test/set_01/test_01_known_vehicle_known_attack/standstill-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_01_known_vehicle_known_attack/standstill-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_02_unknown_vehicle_known_attack/standstill-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_02_unknown_vehicle_known_attack/standstill-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_03_known_vehicle_unknown_attack/standstill-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_03_known_vehicle_unknown_attack/standstill-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_04_unknown_vehicle_unknown_attack/standstill-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_04_unknown_vehicle_unknown_attack/standstill-4.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/standstill-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/standstill-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/standstill-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/standstill-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/standstill-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/standstill-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/standstill-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/standstill-2.csv'
]

In [3]:
# Function to Load Data
def load_data_from_directory(directory_path):
    data_frames = []
    for file_name in os.listdir(directory_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(directory_path, file_name)
            df = pd.read_csv(file_path)
            data_frames.append(df)
    return pd.concat(data_frames, ignore_index=True)

# Function to Preprocess Data
def hex_to_int(x):
    try:
        return int(str(x), 16)
    except ValueError:
        return 0  # Handle non-hexadecimal values
        
def preprocess_data(df):
    df['timestamp'] = pd.to_datetime(df['timestamp']).astype(np.int64) // 10**9
    df['arbitration_id'] = df['arbitration_id'].apply(hex_to_int)
    df['data_field'] = df['data_field'].apply(hex_to_int)
    return df

# Function to Extract Features and Labels
def extract_features_labels(df, label_col='attack'):
    X = df.drop(columns=label_col)
    y = df[label_col]
    return X, y

# Function to load and combine attack files
def load_combine_attack_files(files):
    data_frames = []
    for file in files:
        df = pd.read_csv(file)
        data_frames.append(df)
    combined_df = pd.concat(data_frames, ignore_index=True)
    return combined_df

In [4]:
# Function to Train Isolation Forest Model
def train_isolation_forest(X_train, y_train):
    # Apply SMOTE for Handling Class Imbalance
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Standardize the Features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_resampled)

    # Train the Isolation Forest Model
    iso_forest = IsolationForest(contamination=0.1, random_state=42)

    # Record training start time
    start_time = time.time()
    
    iso_forest.fit(X_train_scaled)

    # Calculate training time
    training_time = time.time() - start_time
    
    return iso_forest, scaler, training_time

# Function to Test Isolation Forest Model
def test_isolation_forest(iso_forest, scaler, X_test, y_test):
    # Standardize the Features
    X_test_scaled = scaler.transform(X_test)

    # Record testing start time
    start_time = time.time()
    # Predict Outliers on Test Data
    test_predictions = iso_forest.predict(X_test_scaled)

    # Calculate testing time
    testing_time = time.time() - start_time
    
    test_labels = np.where(test_predictions == -1, 1, 0)  # Convert -1 to 1 (attack) and 1 to 0 (normal)

    # Evaluate Test Performance
    conf_matrix_test = confusion_matrix(y_test, test_labels)
    mcc_test = matthews_corrcoef(y_test, test_labels)
    accuracy_test = accuracy_score(y_test, test_labels)
    precision_test = precision_score(y_test, test_labels, average='weighted', zero_division=0)
    recall_test = recall_score(y_test, test_labels, average='weighted', zero_division=0)
    f1_test = f1_score(y_test, test_labels, average='weighted', zero_division=0)
    informedness_test = recall_test - (1 - recall_test)
    markedness_test = precision_test - (1 - precision_test)

    return {
        'conf_matrix': conf_matrix_test,
        'mcc': mcc_test,
        'accuracy': accuracy_test,
        'precision': precision_test,
        'recall': recall_test,
        'f1_score': f1_test,
        'informedness': informedness_test,
        'markedness': markedness_test,
        'testing_time': testing_time
    }


In [5]:
# Load and preprocess training data
df_train = pd.concat([load_data_from_directory(train_dir) for train_dir in train_directories], ignore_index=True)
df_train = preprocess_data(df_train)
X_train, y_train = extract_features_labels(df_train)

# Train the Isolation Forest Model
iso_forest, scaler, training_time = train_isolation_forest(X_train, y_train)

In [6]:
# Load and preprocess testing data - DOS attack
df_test = pd.concat([pd.read_csv(file) for file in dos_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Test the Isolation Forest Model
test_metrics = test_isolation_forest(iso_forest, scaler, X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (DOS Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (DOS Attack Test Files):

-----------------------------------------------------------------------

Training Time: 2317.32 seconds
Testing Time: 190.18 seconds

-----------------------------------------------------------------------
Accuracy: 86.61%
Precision: 89.77%
Recall: 86.61%
F1-Score: 88.16%
Matthews Correlation Coefficient: -0.0539
Informedness: 0.7321
Markedness: 0.7955


In [7]:
# Load and preprocess testing data - Force Neutral
df_test = pd.concat([pd.read_csv(file) for file in fn_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Test the Isolation Forest Model
test_metrics = test_isolation_forest(iso_forest, scaler, X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (Force neutral Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Force neutral Attack Test Files):

-----------------------------------------------------------------------

Training Time: 2317.32 seconds
Testing Time: 350.10 seconds

-----------------------------------------------------------------------
Accuracy: 93.78%
Precision: 99.82%
Recall: 93.78%
F1-Score: 96.71%
Matthews Correlation Coefficient: -0.0076
Informedness: 0.8757
Markedness: 0.9963


In [8]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in rpm_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Test the Isolation Forest Model
test_metrics = test_isolation_forest(iso_forest, scaler, X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (RPM Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (RPM Attack Test Files):

-----------------------------------------------------------------------

Training Time: 2317.32 seconds
Testing Time: 317.23 seconds

-----------------------------------------------------------------------
Accuracy: 93.28%
Precision: 99.76%
Recall: 93.28%
F1-Score: 96.41%
Matthews Correlation Coefficient: -0.0091
Informedness: 0.8655
Markedness: 0.9951


In [9]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in standstill_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Test the Isolation Forest Model
test_metrics = test_isolation_forest(iso_forest, scaler, X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (Standstill Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Standstill Attack Test Files):

-----------------------------------------------------------------------

Training Time: 2317.32 seconds
Testing Time: 228.16 seconds

-----------------------------------------------------------------------
Accuracy: 91.54%
Precision: 99.52%
Recall: 91.54%
F1-Score: 95.36%
Matthews Correlation Coefficient: -0.0144
Informedness: 0.8308
Markedness: 0.9903


In [10]:
# Test files for DoS attack
double_files = [
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/double-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/double-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/double-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/double-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/double-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/double-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/double-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/double-4.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/double-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/double-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/double-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/double-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/double-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/double-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/double-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/double-2.csv'
]

# Test files for Force neutral attack
triple_files = [
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/triple-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/triple-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/triple-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/triple-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/triple-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/triple-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/triple-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/triple-4.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/triple-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/triple-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/triple-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/triple-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/triple-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/triple-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/triple-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/triple-2.csv'
]

fuzzing_files = [
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/fuzzing-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/fuzzing-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/fuzzing-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/fuzzing-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/fuzzing-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/fuzzing-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/fuzzing-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/fuzzing-4.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/fuzzing-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/fuzzing-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/fuzzing-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/fuzzing-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/fuzzing-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/fuzzing-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/fuzzing-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/fuzzing-2.csv'
]

interval_files = [
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/interval-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/interval-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/interval-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/interval-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/interval-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/interval-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/interval-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/interval-4.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/interval-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/interval-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/interval-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/interval-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/interval-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/interval-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/interval-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/interval-2.csv'
]

speed_files = [
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/speed-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/speed-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/speed-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/speed-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/speed-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/speed-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/speed-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/speed-4.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/speed-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/speed-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/speed-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/speed-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/speed-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/speed-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/speed-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/speed-2.csv'
]

systematic_files = [
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/systematic-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/systematic-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/systematic-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/systematic-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/systematic-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/systematic-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/systematic-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/systematic-4.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/systematic-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/systematic-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/systematic-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/systematic-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/systematic-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/systematic-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/systematic-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/systematic-2.csv'
]

rpmaccessory_files = [
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/rpm-accessory-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/rpm-accessory-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/rpm-accessory-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/rpm-accessory-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/rpm-accessory-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/rpm-accessory-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/rpm-accessory-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/rpm-accessory-2.csv'
]

speedaccessory_files = [
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/speed-accessory-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/speed-accessory-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/speed-accessory-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/speed-accessory-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/speed-accessory-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/speed-accessory-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/speed-accessory-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/speed-accessory-2.csv'
]

In [11]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in double_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Test the Isolation Forest Model
test_metrics = test_isolation_forest(iso_forest, scaler, X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (Double Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Double Attack Test Files):

-----------------------------------------------------------------------

Training Time: 2317.32 seconds
Testing Time: 310.88 seconds

-----------------------------------------------------------------------
Accuracy: 91.67%
Precision: 99.23%
Recall: 91.67%
F1-Score: 95.30%
Matthews Correlation Coefficient: -0.0180
Informedness: 0.8334
Markedness: 0.9845


In [12]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in triple_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Test the Isolation Forest Model
test_metrics = test_isolation_forest(iso_forest, scaler, X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (Triple Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Triple Attack Test Files):

-----------------------------------------------------------------------

Training Time: 2317.32 seconds
Testing Time: 305.31 seconds

-----------------------------------------------------------------------
Accuracy: 92.09%
Precision: 97.52%
Recall: 92.09%
F1-Score: 94.73%
Matthews Correlation Coefficient: -0.0293
Informedness: 0.8419
Markedness: 0.9503


In [13]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in fuzzing_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Test the Isolation Forest Model
test_metrics = test_isolation_forest(iso_forest, scaler, X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (Fuzzing Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Fuzzing Attack Test Files):

-----------------------------------------------------------------------

Training Time: 2317.32 seconds
Testing Time: 230.18 seconds

-----------------------------------------------------------------------
Accuracy: 90.74%
Precision: 96.43%
Recall: 90.74%
F1-Score: 93.27%
Matthews Correlation Coefficient: 0.1714
Informedness: 0.8148
Markedness: 0.9285


In [14]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in interval_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Test the Isolation Forest Model
test_metrics = test_isolation_forest(iso_forest, scaler, X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (Interval Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Interval Attack Test Files):

-----------------------------------------------------------------------

Training Time: 2317.32 seconds
Testing Time: 268.67 seconds

-----------------------------------------------------------------------
Accuracy: 91.05%
Precision: 96.30%
Recall: 91.05%
F1-Score: 93.60%
Matthews Correlation Coefficient: -0.0376
Informedness: 0.8210
Markedness: 0.9259


In [15]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in speed_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Test the Isolation Forest Model
test_metrics = test_isolation_forest(iso_forest, scaler, X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (Speed Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Speed Attack Test Files):

-----------------------------------------------------------------------

Training Time: 2317.32 seconds
Testing Time: 440.66 seconds

-----------------------------------------------------------------------
Accuracy: 93.83%
Precision: 99.60%
Recall: 93.83%
F1-Score: 96.63%
Matthews Correlation Coefficient: -0.0111
Informedness: 0.8767
Markedness: 0.9920


In [16]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in systematic_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Test the Isolation Forest Model
test_metrics = test_isolation_forest(iso_forest, scaler, X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (Systematic Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Systematic Attack Test Files):

-----------------------------------------------------------------------

Training Time: 2317.32 seconds
Testing Time: 193.72 seconds

-----------------------------------------------------------------------
Accuracy: 89.54%
Precision: 98.69%
Recall: 89.54%
F1-Score: 93.76%
Matthews Correlation Coefficient: 0.0823
Informedness: 0.7909
Markedness: 0.9737


In [17]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in rpmaccessory_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Test the Isolation Forest Model
test_metrics = test_isolation_forest(iso_forest, scaler, X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (RPM Accessory Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (RPM Accessory Attack Test Files):

-----------------------------------------------------------------------

Training Time: 2317.32 seconds
Testing Time: 32.76 seconds

-----------------------------------------------------------------------
Accuracy: 91.11%
Precision: 99.56%
Recall: 91.11%
F1-Score: 95.15%
Matthews Correlation Coefficient: -0.0141
Informedness: 0.8222
Markedness: 0.9913


In [18]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in speedaccessory_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Test the Isolation Forest Model
test_metrics = test_isolation_forest(iso_forest, scaler, X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (Speed Accessory Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Speed Accessory Attack Test Files):

-----------------------------------------------------------------------

Training Time: 2317.32 seconds
Testing Time: 39.33 seconds

-----------------------------------------------------------------------
Accuracy: 89.50%
Precision: 99.24%
Recall: 89.50%
F1-Score: 94.12%
Matthews Correlation Coefficient: -0.0201
Informedness: 0.7901
Markedness: 0.9849
